# 추천 시스템 - 중요⭐

Data가 없는 경우  
-> 고객의 행동에서 특정 콘텐츠의 특징을 추출하여 다음 추천을 결정(콘텐츠 기반 필터링) - 정확도는 떨어짐     
  
Data가 쌓인 경우  
-> 고객들간의 지난 행동이 유사한 것이라 가정하여 다른 사람의 패턴과 유사함을 비교하여 추천(협업 필터링)

# 콘텐츠 기반 필터링 예측

In [2]:
import pandas as pd

raw_df = pd.read_csv('../data/tmdb_5000_movies.csv')
raw_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
df = raw_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'popularity', 'overview']]
pd.set_option('max_colwidth', 20)
df

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,"[{""id"": 28, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""n...",150.437577,In the 22nd cent...
1,285,Pirates of the C...,"[{""id"": 12, ""nam...",6.9,4500,139.082615,"[{""id"": 270, ""na...",139.082615,Captain Barbossa...
2,206647,Spectre,"[{""id"": 28, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""na...",107.376788,A cryptic messag...
3,49026,The Dark Knight ...,"[{""id"": 28, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""na...",112.312950,Following the de...
4,49529,John Carter,"[{""id"": 28, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""na...",43.926995,John Carter is a...
...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""nam...",6.6,238,14.269792,"[{""id"": 5616, ""n...",14.269792,El Mariachi just...
4799,72766,Newlyweds,"[{""id"": 35, ""nam...",5.9,5,0.642552,[],0.642552,A newlywed coupl...
4800,231617,"Signed, Sealed, ...","[{""id"": 35, ""nam...",7.0,6,1.444476,"[{""id"": 248, ""na...",1.444476,"""Signed, Sealed,..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],0.857008,When ambitious N...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   popularity    4803 non-null   float64
 8   overview      4800 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 337.8+ KB


In [5]:
from ast import literal_eval # 문자열을 자료형으로 변환(str -> list)

# 문자열 -> list, dict로 변환
# result = literal_eval(df.iloc[0]['genres']) # 각각 적용하여 결과 추출

df['genres'] = df['genres'].apply(literal_eval) # 함수 자체를 해당 컬럼에 적용
df['genres']

C:\Users\user\AppData\Local\Temp\ipykernel_47752\523635665.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(literal_eval) # 함수 자체를 해당 컬럼에 적용


0       [{'id': 28, 'nam...
1       [{'id': 12, 'nam...
2       [{'id': 28, 'nam...
3       [{'id': 28, 'nam...
4       [{'id': 28, 'nam...
               ...         
4798    [{'id': 28, 'nam...
4799    [{'id': 35, 'nam...
4800    [{'id': 35, 'nam...
4801                     []
4802    [{'id': 99, 'nam...
Name: genres, Length: 4803, dtype: object

In [6]:
df['keywords'] = df['keywords'].apply(literal_eval)
df['keywords']

C:\Users\user\AppData\Local\Temp\ipykernel_47752\3086860884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].apply(literal_eval)


0       [{'id': 1463, 'n...
1       [{'id': 270, 'na...
2       [{'id': 470, 'na...
3       [{'id': 849, 'na...
4       [{'id': 818, 'na...
               ...         
4798    [{'id': 5616, 'n...
4799                     []
4800    [{'id': 248, 'na...
4801                     []
4802    [{'id': 1523, 'n...
Name: keywords, Length: 4803, dtype: object

In [7]:
total_names = []

for id_names in df['genres']:
    
    names = []
    
    for id_name in id_names:
        names.append(id_name['name'])
        
    total_names.append(' '.join(names))   
    
df['genres'] = total_names

C:\Users\user\AppData\Local\Temp\ipykernel_47752\1897628688.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = total_names


In [8]:
total_keywords = []

for id_keywords in df['keywords']:
    
    keywords = []
    
    for id_keyword in id_keywords:
        keywords.append(id_keyword['name'])
        
    total_keywords.append(' '.join(keywords))   
    
df['keywords'] = total_keywords 

C:\Users\user\AppData\Local\Temp\ipykernel_47752\3119218250.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = total_keywords


In [9]:
# pandas select
print(df.loc[3, 'genres']) # 값
print(df.iloc[3]['genres']) # 위치

Action Crime Drama Thriller
Action Crime Drama Thriller


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(min_df=1, ngram_range=(1,2)) # 최소 1번 이상 나오고, 2개씩 조합한 경우의 수를 벡터 행으로 활용

genres_vector = count.fit_transform(df['genres']) # 'comedy action'
keywords_vector = count.fit_transform(df['keywords']) # 'summer car'

count.vocabulary_ # 컬럼의 순서(횟수가 아님)

{'culture': 9037,
 'clash': 7010,
 'future': 15032,
 'space': 34485,
 'war': 40021,
 'colony': 7433,
 'society': 34214,
 'travel': 38236,
 'futuristic': 15087,
 'romance': 31412,
 'alien': 1077,
 'tribe': 38408,
 'planet': 27732,
 'cgi': 6161,
 'marine': 22347,
 'soldier': 34275,
 'battle': 3232,
 'love': 21496,
 'affair': 548,
 'anti': 1760,
 'power': 28328,
 'relations': 30072,
 'mind': 23178,
 'and': 1426,
 'soul': 34431,
 '3d': 94,
 'culture clash': 9039,
 'clash future': 7012,
 'future space': 15075,
 'space war': 34525,
 'war space': 40145,
 'space colony': 34494,
 'colony society': 7442,
 'society space': 34240,
 'space travel': 34523,
 'travel futuristic': 38253,
 'futuristic romance': 15094,
 'romance space': 31433,
 'space alien': 34487,
 'alien tribe': 1148,
 'tribe alien': 38409,
 'alien planet': 1122,
 'planet cgi': 27737,
 'cgi marine': 6165,
 'marine soldier': 22373,
 'soldier battle': 34281,
 'battle love': 3254,
 'love affair': 21498,
 'affair anti': 550,
 'anti war': 

In [11]:
genres_vector.shape # 전체 영화수, 장르의 조합수(벡터의 열수)

(4803, 276)

In [12]:
keywords_vector.shape # 전체 영화수, 키워드의 조합수(벡터의 열수)

(4803, 41554)

In [13]:
genres_vector_array = genres_vector.toarray() # 각 행이 각 영화
genres_vector_array

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 276))

In [14]:
keywords_vector_array = keywords_vector.toarray()
keywords_vector_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 41554))

In [15]:
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도로 유사한 행 확인

genre_sim = cosine_similarity(genres_vector_array, genres_vector_array)
keyword_sim = cosine_similarity(keywords_vector_array, keywords_vector_array)

In [16]:
df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,Action Adventure...,7.2,11800,150.437577,culture clash fu...,150.437577,In the 22nd cent...
1,285,Pirates of the C...,Adventure Fantas...,6.9,4500,139.082615,ocean drug abuse...,139.082615,Captain Barbossa...
2,206647,Spectre,Action Adventure...,6.3,4466,107.376788,spy based on nov...,107.376788,A cryptic messag...
3,49026,The Dark Knight ...,Action Crime Dra...,7.6,9106,112.312950,dc comics crime ...,112.312950,Following the de...
4,49529,John Carter,Action Adventure...,6.1,2124,43.926995,based on novel m...,43.926995,John Carter is a...


## 추천 함수

In [17]:
# 컨텐츠 기반 추출방식은 지양하는 것이 요즘 추세
def recommend_movies_by_genre(df, genre_sorted_index, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_index = genre_sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]
    
genre_sim_sorted = genre_sim.argsort()[:,::-1] # 기본값이 오름차순이라 역순으로 바꾼 후에 나오는 index 활용

recommend_movies_by_genre(df, genre_sim_sorted, 'Superman', 5)

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,Action Adventure...,7.2,11800,150.437577,culture clash fu...,150.437577,In the 22nd cent...
46,127585,X-Men: Days of F...,Action Adventure...,7.5,6032,118.078691,1970s mutant tim...,118.078691,The ultimate X-M...
3494,27549,Beastmaster 2: T...,Action Adventure...,4.6,17,1.478505,based on novel t...,1.478505,Mark Singer retu...
870,8536,Superman II,Action Adventure...,6.5,629,30.515175,saving the world...,30.515175,Three escaped cr...
14,49521,Man of Steel,Action Adventure...,6.5,6359,99.398009,saving the world...,99.398009,A young boy lear...


In [18]:
def recommend_movies_by_keyword(df, keyword_sorted_index, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_index = keyword_sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]
    
keyword_sim_sorted = keyword_sim.argsort()[:,::-1]

recommend_movies_by_keyword(df, keyword_sim_sorted, 'Superman', 5)

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
813,1924,Superman,Action Adventure...,6.9,1022,48.507081,saving the world...,48.507081,Mild-mannered Cl...
10,1452,Superman Returns,Adventure Fantas...,5.4,1400,57.925623,saving the world...,57.925623,Superman returns...
1296,9531,Superman III,Comedy Action Ad...,5.3,490,22.164202,saving the world...,22.164202,Aiming to defeat...
14,49521,Man of Steel,Action Adventure...,6.5,6359,99.398009,saving the world...,99.398009,A young boy lear...
870,8536,Superman II,Action Adventure...,6.5,629,30.515175,saving the world...,30.515175,Three escaped cr...


# 협업 필터링 예측

In [19]:
import pandas as pd

movies = pd.read_csv('../data/movies.csv')
print(movies.head())
ratings = pd.read_csv('../data/ratings.csv')
ratings.head()


   movieId                title               genres
0        1     Toy Story (1995)  Adventure|Animat...
1        2       Jumanji (1995)  Adventure|Childr...
2        3  Grumpier Old Men...       Comedy|Romance
3        4  Waiting to Exhal...  Comedy|Drama|Rom...
4        5  Father of the Br...               Comedy


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## 협업 필터링(유저 베이스 필터링)

In [20]:
mr = pd.merge(movies, ratings, on='movieId')

In [21]:
title_user = mr.pivot_table(index='userId', columns='title', values='rating') # index가 행

In [ ]:
title_user.fillna(0, inplace=True) # -1로 채우는 경우 벡터의 방향성이 생기므로 0으로 처리

In [23]:
user_based_result = cosine_similarity(title_user, title_user)

In [ ]:
user_based_df = pd.DataFrame(user_based_result, index=title_user.index, columns=title_user.index)
user_based_df

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.903794,0.928342,0.734677,0.866068,0.939770,0.899250,0.847844,0.931406,0.930432,...,0.924628,0.954136,0.535747,0.641526,0.949916,0.907544,0.959599,0.949312,0.945228,0.848348
2,0.903794,1.000000,0.880577,0.705694,0.823569,0.874415,0.865969,0.810361,0.880414,0.871132,...,0.927459,0.894439,0.517661,0.639574,0.934004,0.911802,0.900820,0.883970,0.899107,0.809353
3,0.928342,0.880577,1.000000,0.710558,0.851080,0.904656,0.875089,0.852402,0.904170,0.901827,...,0.902211,0.917410,0.556499,0.645809,0.923934,0.883466,0.926587,0.912233,0.916513,0.837231
4,0.734677,0.705694,0.710558,1.000000,0.685496,0.717615,0.756714,0.695141,0.701059,0.724646,...,0.716820,0.720608,0.459156,0.592987,0.731516,0.699959,0.727642,0.726159,0.717152,0.701400
5,0.866068,0.823569,0.851080,0.685496,1.000000,0.845487,0.813357,0.793942,0.841735,0.832846,...,0.854234,0.851843,0.528025,0.642076,0.863642,0.818104,0.859164,0.847478,0.850166,0.808188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.907544,0.911802,0.883466,0.699959,0.818104,0.878078,0.872043,0.804514,0.877547,0.873551,...,0.912046,0.896596,0.518509,0.629157,0.924740,1.000000,0.904335,0.886625,0.900852,0.815343
668,0.959599,0.900820,0.926587,0.727642,0.859164,0.930343,0.887337,0.850815,0.933824,0.926979,...,0.919436,0.946171,0.544503,0.641923,0.945186,0.904335,1.000000,0.935546,0.944965,0.851949
669,0.949312,0.883970,0.912233,0.726159,0.847478,0.918218,0.877799,0.833750,0.909613,0.912829,...,0.902950,0.932096,0.536234,0.640701,0.928156,0.886625,0.935546,1.000000,0.924635,0.836524


In [25]:
user_based_df.iloc[0].sort_values(ascending=False)[1:6]

userId
325    0.981684
579    0.977799
35     0.975080
310    0.974584
634    0.972941
Name: 1, dtype: float64

In [26]:
# 1. 내가 본것 

my_list = []

ratings = title_user.iloc[0]

for idx, rating in enumerate(ratings): # index / 평점
    
    if rating > 0:
        my_list.append(title_user.iloc[0].index[idx])
    
print(my_list)    
print(len(my_list))

# 2. 상대가 본 것 중에 평점 4.0이상

you_list = []

# loc(값비교) iloc(index비교) 
# 325를 넣었을때 => iloc로 324번째 값 / loc는 userId가 325인 값
ratings = title_user.loc[325] 
you_list = ratings[ratings>=4].index

print(you_list)
print(len(you_list))

['Antz (1998)', 'Beavis and Butt-Head Do America (1996)', 'Ben-Hur (1959)', 'Blazing Saddles (1974)', 'Cape Fear (1991)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 'Dangerous Minds (1995)', 'Deer Hunter, The (1978)', "Dracula (Bram Stoker's Dracula) (1992)", 'Dumbo (1941)', 'Escape from New York (1981)', 'Fly, The (1986)', 'French Connection, The (1971)', 'Gandhi (1982)', 'Gods Must Be Crazy, The (1980)', 'Sleepers (1996)', 'Star Trek: The Motion Picture (1979)', 'Time Bandits (1981)', 'Tron (1982)', 'Willow (1988)']
20
Index(['Beverly Hills Cop (1984)', 'Brady Bunch Movie, The (1995)',
       'Dangerous Minds (1995)', 'My Best Friend's Wedding (1997)'],
      dtype='object', name='title')
4


In [27]:
# 3. 차집합으로 상대방은 봤는데 내가 안본 영화 확인
set(you_list) - set(my_list)

{'Beverly Hills Cop (1984)',
 'Brady Bunch Movie, The (1995)',
 "My Best Friend's Wedding (1997)"}

## 협업 필터링(콘텐츠 베이스 필터링)

In [30]:
content_base = title_user.T # 기존 데이터의 행렬 전환

print(content_base)

content_based_sim = cosine_similarity(content_base, content_base)
content_based_df = pd.DataFrame(content_based_sim, index=content_base.index, columns=content_base.index)

content_based_df


userId                1    2    3    4    5    6    7    8    9    10   ...  \
title                                                                   ...   
"Great Performanc... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
$9.99 (2008)         -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
'Hellboy': The Se... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
'Neath the Arizon... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
'Round Midnight (... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
...                   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
xXx (2002)           -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
xXx: State of the... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
¡Three Amigos! (1... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
À nous la liberté... -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ...   
İtirazım Var (2014)  -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.000000,0.934994,0.979853,0.988152,0.969943,0.971706,0.947214,0.778514,0.912117,0.483226,...,0.950646,0.981868,0.945284,0.719699,0.964597,0.784622,0.983524,0.649423,0.964597,0.971706
$9.99 (2008),0.934994,1.000000,0.941540,0.946337,0.928778,0.933681,0.906821,0.764819,0.876172,0.504960,...,0.913340,0.943486,0.908131,0.689943,0.926831,0.759430,0.945086,0.638553,0.926831,0.933681
'Hellboy': The Seeds of Creation (2004),0.979853,0.941540,1.000000,0.991635,0.973364,0.998359,0.950559,0.800207,0.918521,0.485014,...,0.957275,0.988673,0.951893,0.725150,0.971299,0.790453,0.990337,0.654521,0.971299,0.978451
'Neath the Arizona Skies (1934),0.988152,0.946337,0.991635,1.000000,0.981617,0.983403,0.958685,0.788470,0.923212,0.490386,...,0.962138,0.993670,0.956738,0.729093,0.976217,0.794683,0.995340,0.658186,0.976217,0.983403
'Round Midnight (1986),0.969943,0.928778,0.973364,0.981617,1.000000,0.965267,0.940921,0.792953,0.906045,0.503531,...,0.944335,0.975367,0.939002,0.733708,0.958203,0.779271,0.977012,0.644887,0.958203,0.965267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xXx (2002),0.784622,0.759430,0.790453,0.794683,0.779271,0.783655,0.759786,0.668526,0.733650,0.456876,...,0.792217,0.798884,0.761085,0.581687,0.777772,1.000000,0.801649,0.591101,0.777772,0.810712
xXx: State of the Union (2005),0.983524,0.945086,0.990337,0.995340,0.977012,0.982114,0.954140,0.787364,0.921988,0.496804,...,0.960871,0.992370,0.955475,0.728047,0.974937,0.801649,1.000000,0.657206,0.974937,0.982114
¡Three Amigos! (1986),0.649423,0.638553,0.654521,0.658186,0.644887,0.648731,0.647065,0.664333,0.668747,0.337855,...,0.657358,0.668210,0.629145,0.558277,0.643754,0.591101,0.657206,1.000000,0.643754,0.648731


In [ ]:
# 유저 기반으로 나온 결과(영화명)를 가지고, 컨텐츠 기반으로 다시 추천하는 것도 방법
content_based_df['Toy Story (1995)'].sort_values(ascending=False)[1:6] 

title
Toy Story 2 (1999)                           0.428382
Star Wars: Episode IV - A New Hope (1977)    0.414637
Independence Day (a.k.a. ID4) (1996)         0.407789
Forrest Gump (1994)                          0.385906
Groundhog Day (1993)                         0.382651
Name: Toy Story (1995), dtype: float64

# 서프라이즈 활용 평점 예측
### 사용자가 미사용한 상품에 대해 평점을 예상하고 높은 순으로 추천

In [56]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

import pandas as pd

In [11]:
ratings = pd.read_csv('../data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [16]:
ratings.to_csv('../data/ratings_nohead.csv', index=False, header=False)
ratings.describe() # 최소는 0이 아니라 0.5


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [17]:
ratings['rating'].unique()

array([2.5, 3. , 2. , 4. , 3.5, 1. , 5. , 4.5, 1.5, 0.5])

In [21]:
# line_format(nohead로 기존 컬럼명 제거했기 때문에 재설정), sep(구분자 : 기본값 ',')
movie_reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5)) 
data_folds = DatasetAutoFolds(ratings_file='../data/ratings_nohead.csv', reader=movie_reader)

# 100% 다 훈련용으로 사용
trainset = data_folds.build_full_trainset() 

model = SVD(n_factors=50, random_state=42) # n_factors - 차원수
model.fit(trainset)

In [ ]:
y_pred = model.predict('9', '3') # 9번 유저가 3번 영화를 보면 3.09점(est)을 줄 것으로 예상
y_pred

Prediction(uid='9', iid='3', r_ui=None, est=3.092324541202098, details={'was_impossible': False})

In [32]:
# 1. 해당 유저가 안본 영화를 확인
movies = pd.read_csv('../data/movies.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9125 non-null   int64 
 1   title    9125 non-null   object
 2   genres   9125 non-null   object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB


In [37]:
# 해당 유저가 본 영화
ratings[ratings['userId'] == 9]['movieId']
movies['movieId']

0            1
1            2
2            3
3            4
4            5
         ...  
9120    162672
9121    163056
9122    163949
9123    164977
9124    164979
Name: movieId, Length: 9125, dtype: int64

In [38]:
def get_unseen_movies(userId):
    return list(set(movies['movieId']) - set(ratings[ratings['userId'] == userId]['movieId']))

unseen_movies = get_unseen_movies(9)

In [ ]:
pred_list = []

for movieId in unseen_movies:
    y_pred = model.predict('9', str(movieId))
    pred_list.append(
        {
            'movieId': y_pred.iid, 
            'rating': y_pred.est, 
            'title': movies[movies['movieId'] == int(y_pred.iid)]['title'].iloc[0] # TODO 왜 0? 
        }
    )

print(pred_list[0])

{'movieId': '2', 'rating': 3.4150971107531425, 'title': 'Jumanji (1995)'}


In [55]:
result_df = pd.DataFrame(pred_list)
result_df.sort_values(by='rating', ascending=False)[:5]

,movieId,rating,title
849,969,4.498787,"African Queen, The (1951)"
752,858,4.453779,"Godfather, The (1972)"
795,913,4.450571,"Maltese Falcon, The (1941)"
1066,1221,4.434328,"Godfather: Part II, The (1974)"
2983,3462,4.390097,Modern Times (1936)
